In [1]:
from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning as pl
import sklearn 
from sklearn.metrics import confusion_matrix, auc, roc_curve
import torch
import pandas as pd
import pickle
from py_jotools import mut, slurm, cache
import numpy as np
import gc

import hashlib
import dill

from sklearn.metrics import mean_absolute_error
from datasets.ContinuousDataset import BrainAgeContinuous
import yaml
import active_dynamicmemory.runutils as rutils
%load_ext autoreload
%autoreload 

In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mperkonigg/.local/lib/python3.6/site-packages/m

In [2]:
def train_config(configfile, remote=False, runs=None, jobarray=False):
    with open(configfile) as f:
        params = yaml.load(f, Loader=yaml.FullLoader)

    if remote:
        with open('training_configs/slurm_config.yml') as s:
            sparams = yaml.load(s, Loader=yaml.FullLoader)

    if jobarray:
        sparams['qos'] = 'jobarray'

    if runs is None:
        if remote:
            print('scheduling job to CIR cluster...')
            slurm.srun(rutils.trained_model, [params['trainparams'], params['settings']], params=sparams, remote=True)
        else:
            model, logs, df_mem, exp_name = rutils.trained_model(params['trainparams'], params['settings'])
            print('successfully trained model', exp_name)
    else:
        for i in range(runs):
            params['trainparams']['seed'] = i+1
            params['trainparams']['run_postfix'] = i+1
            if remote:
                print('scheduling job to CIR cluster...')
                slurm.srun(rutils.trained_model, [params['trainparams'], params['settings']], params=sparams, remote=True)
            else:
                model, logs, df_mem, exp_name = rutils.trained_model(params['trainparams'], params['settings'])
                print('successfully trained model', exp_name)

In [3]:
def train_params(params, remote=False, runs=None, jobarray=False):
    if remote:
        with open('training_configs/slurm_config.yml') as s:
            sparams = yaml.load(s, Loader=yaml.FullLoader)

    if jobarray:
        sparams['qos'] = 'jobarray'

    if runs is None:
        if remote:
            print('scheduling job to CIR cluster...')
            slurm.srun(rutils.trained_model, [params['trainparams'], params['settings']], params=sparams, remote=True)
        else:
            model, logs, df_mem, exp_name = rutils.trained_model(params['trainparams'], params['settings'])
            print('successfully trained model', exp_name)
    else:
        for i in range(runs):
            params['trainparams']['seed'] = i+1
            params['trainparams']['run_postfix'] = i+1
            if remote:
                print('scheduling job to CIR cluster...')
                slurm.srun(rutils.trained_model, [params['trainparams'], params['settings']], params=sparams, remote=True)
            else:
                model, logs, df_mem, exp_name = rutils.trained_model(params['trainparams'], params['settings'])
                print('successfully trained model', exp_name)

In [2]:
pd.read_csv('/project/catinous/brainds_split_val_test_melba.csv').groupby('split').count()

,ID,scanner,Age,Image
split,,,,
base,201,201,201,201
test,259,259,259,259
train,1892,1892,1892,1892
val,259,259,259,259


In [3]:
1892/20, 1892/10, 1892/8, 1892/5

(94.6, 189.2, 236.5, 378.4)

In [4]:
with open(f'./training_configs/brainage_naive_10.yml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)
    
for seed in [6, 7, 8, 9, 10]:
    params['trainparams']['seed'] = seed
    params['trainparams']['run_postfix'] = seed
    
    train_params(params, remote=True)

scheduling job to CIR cluster...
adm75404
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm75404.job
Submitted batch job 3568822

scheduling job to CIR cluster...
adm84097
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm84097.job
Submitted batch job 3568823

scheduling job to CIR cluster...
adm40598
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm40598.job
Submitted batch job 3568824

scheduling job to CIR cluster...
adm51435
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm51435.job
Submitted batch job 3568825

scheduling job to CIR cluster...
adm53017
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm53017.job
Submitted batch job 3568826



In [6]:
train_config('training_configs/brainage_base.yml', remote=True)

scheduling job to CIR cluster...
adm55554
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm55554.job
Submitted batch job 3493097



In [8]:
with open(f'./training_configs/brainage_15phil.yml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)

for scanner in ['1.5T Philips', '1.5T', '3.0T', '3.0T Philips']:
    params['trainparams']['scanner'] = scanner
    
    train_params(params, remote=True, runs=5, jobarray=True)

scheduling job to CIR cluster...
adm47541
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm47541.job
Submitted batch job 3568354

scheduling job to CIR cluster...
adm52768
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm52768.job
Submitted batch job 3568355

scheduling job to CIR cluster...
adm94225
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm94225.job
Submitted batch job 3568356

scheduling job to CIR cluster...
adm13633
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm13633.job
Submitted batch job 3568357

scheduling job to CIR cluster...
adm30024
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm30024.job
Submitted batch job 3568

In [18]:
with open('training_configs/brainage_naive.yml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)

for ni, sb in zip([5, 8, 10, 20], [378, 236, 189, 95]):
    params['trainparams']['naive_insert_rate'] = ni
    params['trainparams']['startbudget'] = sb
    
    train_params(params, remote=True, runs=5, jobarray=True)

scheduling job to CIR cluster...
adm65356
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm65356.job
Submitted batch job 3442592

scheduling job to CIR cluster...
adm49065
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm49065.job
Submitted batch job 3442593

scheduling job to CIR cluster...
adm38372
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm38372.job
Submitted batch job 3442594

scheduling job to CIR cluster...
adm38279
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm38279.job
Submitted batch job 3442595

scheduling job to CIR cluster...
adm51003
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm51003.job
Submitted batch job 3442

In [6]:
with open('training_configs/brainage_naive.yml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)
    
train_params(params, remote=True, runs=5, jobarray=True)

scheduling job to CIR cluster...
adm21982
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm21982.job
Submitted batch job 3487798

scheduling job to CIR cluster...
adm13094
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm13094.job
Submitted batch job 3487799

scheduling job to CIR cluster...
adm51872
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm51872.job
Submitted batch job 3487800

scheduling job to CIR cluster...
adm80036
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm80036.job
Submitted batch job 3487801

scheduling job to CIR cluster...
adm35627
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm35627.job
Submitted batch job 3487

In [ ]:
train_config('training_configs/brainage_casa.yml', remote=False)

In [16]:
df_ds = pd.read_csv('/project/catinous/lungnodulesfinallndbBig.csv')

In [19]:
df_ds.groupby(['split']).count()

,dataset,scanner,patient_id,image,x1,x2,y1,y2,bin_malignancy
split,,,,,,,,,
base,253,253,253,253,253,253,253,253,253
test,220,220,220,220,220,220,220,220,220
train,883,883,883,883,883,883,883,883,883
val,141,141,141,141,141,141,141,141,141


In [4]:
for lb in [8, 10, 20]:
    with open(f'./training_configs/brainage_casa_{lb}.yml') as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
    train_params(params, remote=True, runs=5, jobarray=True)
    
for lb in [5, 8, 10, 20]:
    with open(f'./training_configs/brainage_naive_{lb}.yml') as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
    train_params(params, remote=True, runs=5, jobarray=True)

scheduling job to CIR cluster...
adm46595
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm46595.job
Submitted batch job 3567613

scheduling job to CIR cluster...
adm65904
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm65904.job
Submitted batch job 3567614

scheduling job to CIR cluster...
adm04875
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm04875.job
Submitted batch job 3567615

scheduling job to CIR cluster...
adm58493
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm58493.job
Submitted batch job 3567616

scheduling job to CIR cluster...
adm39269
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm39269.job
Submitted batch job 3567